## COCO file -> csv
not yet

In [6]:
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
from pycocotools.coco import COCO
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import skimage.io as io
import pylab
import os
pylab.rcParams['figure.figsize'] = (8.0, 8.0)

import os.path as osp
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset, replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame

In [3]:
gt = []

coco = COCO("best_predictions.json")
   
'''
image id 별로 GT 값 추가 (여기서는 test.json을 사용하였기 때문에 GT가 없습니다.)
'''
for image_id in coco.getImgIds():
        
    image_info = coco.loadImgs(image_id)[0]
    annotation_id = coco.getAnnIds(imgIds=image_info['id'])
    annotation_info_list = coco.loadAnns(annotation_id)
        
    file_name = image_info['file_name']
        
    for annotation in annotation_info_list:
        gt.append([file_name, annotation['category_id'],
                    float(annotation['socre']),
                    float(annotation['bbox'][0]),
                    float(annotation['bbox'][0]) + float(annotation['bbox'][2]),
                    float(annotation['bbox'][1]),
                    (float(annotation['bbox'][1]) + float(annotation['bbox'][3]))])

loading annotations into memory...


AssertionError: annotation file format <class 'list'> not supported

---
## JSON -> Pandas -> CSV

In [1]:
import pandas as pd

In [41]:
data = pd.read_json("/opt/ml/baseline/level2-object-detection-level2-cv-19/yolov5/runs/val/exp3/best_predictions.json")
data.sort_values(by=["image_id"],inplace=True)

In [56]:
flag = data.iloc[0].image_id

inf_dict = {"PredictionString":[],"image_id":[]}

PredictionString = ""
for i in data.iterrows():
    if(flag != i[1].image_id):
        inf_dict["PredictionString"].append(PredictionString)
        inf_dict["image_id"].append("test/" + str(10000 + flag)[1:] + ".jpg")
        PredictionString = ""
        flag = i[1].image_id
        
        # if(i[1].image_id == 5):
        #     break

    #set score threshold
    if(i[1].score < 0.001):
        continue

    add_list = [i[1].category_id, i[1].score, i[1].bbox[1], i[1].bbox[2], i[1].bbox[3]]
    add_list = list(map(str,add_list))
    for i in add_list:
        PredictionString += i + " "
    

In [60]:
pd.DataFrame(inf_dict).to_csv('./test.csv',index=False)